## Out of Sample Analysis

Analisi Out-of-Sample del modello SARIMAX base e con tutte le aggiunte.
Confronto sistematico di modelli di forecasting con test statistici avanzati.

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from dieboldmariano import dm_test
from scipy import stats
from itertools import combinations
import json
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Union
from pathlib import Path
from tabulate import tabulate
import time

# Configurazione warnings
warnings.filterwarnings("ignore", category=sm.tools.sm_exceptions.ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# ==================== CONFIGURAZIONE ====================
class Config:
    """Configurazione centralizzata dell'analisi"""
    
    # Percorsi
    PATH_INPUT_DIR_FASE2 = "/Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2"
    FILE_SERIE_STAZIONARIE_IN = os.path.join(PATH_INPUT_DIR_FASE2, "indici_gt_nic_stazionari_fase2.csv")
    PATH_OUTPUT_OOS = "/Users/tommaso/Desktop/tesi-inflation-gt/SARIMAX_modelli/previsioni_out_of_sample_v3_avanzata"
    
    # Nomi colonne
    COL_INFLAZIONE_STAZ = 'NIC_destag_ISTAT_diff1'
    COL_GT_INFLAZIONE_ORIG = 'indice_Inflazione_GT_PCA_SA_diff1'
    COL_GT_TEMATICO_ORIG = 'indice_Tematico_GT_SA_diff1'
    
    # Parametri analisi
    LUNGHEZZA_FINESTRA_INIZIALE = 180
    ORIZZONTI_PREVISIONE = [1, 3, 6, 12]
    SCHEMA_PREVISIONE = "rolling"  # "rolling" o "recursive"
    MIN_OBS_FORECAST_TESTS = 20
    ALPHA_LEVEL = 0.05
    N_BOOTSTRAP_SPA = 1000
    
    # Configurazione modelli
    MODELLI_DA_TESTARE = {
        "Base_Outliers": {
            "order": (1, 0, 1), 
            "seasonal_order": (0, 0, 0, 12),
            "exog_vars_base": ["d_outlier_2022_01_1m", "d_outlier_2022_10_1m"], 
            "gt_vars_lags": {}
        },
        "GT_Tematico_L1": {
            "order": (1, 0, 1), 
            "seasonal_order": (0, 0, 0, 12),
            "exog_vars_base": ["d_outlier_2022_01_1m", "d_outlier_2022_10_1m"],
            "gt_vars_lags": {"indice_Tematico_GT_SA_diff1": 1}
        },
        "GT_Inflazione_L3": {
            "order": (1, 0, 1), 
            "seasonal_order": (0, 0, 0, 12),
            "exog_vars_base": ["d_outlier_2022_01_1m", "d_outlier_2022_10_1m"],
            "gt_vars_lags": {"indice_Inflazione_GT_PCA_SA_diff1": 3}
        },
        "GT_Entrambi": {
            "order": (1, 0, 1), 
            "seasonal_order": (0, 0, 0, 12),
            "exog_vars_base": ["d_outlier_2022_01_1m", "d_outlier_2022_10_1m"],
            "gt_vars_lags": {
                "indice_Inflazione_GT_PCA_SA_diff1": 3, 
                "indice_Tematico_GT_SA_diff1": 1
            }
        }
    }


# ==================== FUNZIONI DI TEST STATISTICO ====================
def clark_west_test(actual: np.ndarray, pred_restricted: np.ndarray, 
                   pred_unrestricted: np.ndarray, horizon: int = 1) -> Tuple[float, float]:
    """
    Test di Clark-West per modelli nested.
    Come confrontare due strategie d'investimento dove una è versione semplificata dell'altra.
    """
    try:
        actual = np.asarray(actual).flatten()
        pred_r = np.asarray(pred_restricted).flatten()
        pred_u = np.asarray(pred_unrestricted).flatten()
        
        # Verifica lunghezze
        if len(actual) != len(pred_r) or len(actual) != len(pred_u):
            return np.nan, np.nan
            
        error_r = actual - pred_r
        error_u = actual - pred_u
        mse_diff = error_r**2 - error_u**2
        adj_diff = (pred_u - pred_r)**2
        cw_stat_series = mse_diff + adj_diff
        
        mean_cw = np.mean(cw_stat_series)
        n = len(cw_stat_series)
        
        if n == 0:
            return np.nan, np.nan
            
        # Correzione Newey-West per autocorrelazione
        bandwidth = max(1, int(4 * (n/100)**(2/9)))
        centered_series = cw_stat_series - mean_cw
        gamma_0 = np.mean(centered_series**2)
        gamma_sum = 0
        
        for j in range(1, min(bandwidth + 1, n)):
            if n - j > 0:
                gamma_j = np.mean(centered_series[j:] * centered_series[:-j])
                weight = 1 - j / (bandwidth + 1)
                gamma_sum += 2 * weight * gamma_j
                
        variance_cw = (gamma_0 + gamma_sum) / n
        
        if variance_cw > 1e-9:
            t_stat = mean_cw / np.sqrt(variance_cw)
            p_value = 1 - stats.norm.cdf(t_stat)
        else:
            t_stat = np.nan
            p_value = np.nan
            
        return t_stat, p_value
        
    except Exception:
        return np.nan, np.nan


def encompassing_test(actual: np.ndarray, pred1: np.ndarray, 
                     pred2: np.ndarray) -> Tuple[float, float, float]:
    """
    Test di encompassing: verifica se un modello ingloba le informazioni dell'altro.
    Come verificare se un analista senior incorpora già tutte le intuizioni di un junior.
    """
    try:
        actual = np.asarray(actual).flatten()
        pred1 = np.asarray(pred1).flatten()
        pred2 = np.asarray(pred2).flatten()
        
        if len(actual) != len(pred1) or len(actual) != len(pred2):
            return np.nan, np.nan, np.nan
            
        errors1 = actual - pred1
        pred_diff = pred1 - pred2
        
        X = sm.add_constant(pred_diff, prepend=True)
        
        if X.shape[0] < X.shape[1] + 1:
            return np.nan, np.nan, np.nan
            
        model = sm.OLS(errors1, X).fit()
        t_stat_beta = model.tvalues[1]
        p_value_beta = model.pvalues[1]
        beta_hat = model.params[1]
        
        return t_stat_beta, p_value_beta, beta_hat
        
    except Exception:
        return np.nan, np.nan, np.nan


def spa_test(actual: np.ndarray, predictions_dict: Dict[str, np.ndarray], 
            benchmark_model_name: str, n_bootstrap: int = 1000) -> Tuple[float, float, Dict[str, float]]:
    """
    Superior Predictive Ability test di Hansen.
    Come un torneo dove verifichi se esiste almeno una strategia che batte significativamente il benchmark.
    """
    try:
        actual = np.asarray(actual).flatten()
        n_obs = len(actual)
        
        if benchmark_model_name not in predictions_dict:
            return np.nan, np.nan, {}
            
        benchmark_pred = np.asarray(predictions_dict[benchmark_model_name]).flatten()
        benchmark_errors_sq = (actual - benchmark_pred)**2
        
        relative_performance_loss_diff = {}
        
        for model_name, predictions in predictions_dict.items():
            if model_name != benchmark_model_name:
                model_pred = np.asarray(predictions).flatten()
                if len(model_pred) == n_obs:
                    model_errors_sq = (actual - model_pred)**2
                    relative_performance_loss_diff[model_name] = benchmark_errors_sq - model_errors_sq
                
        if not relative_performance_loss_diff:
            return 0, 1.0, {}
            
        mean_loss_diffs = {name: np.mean(diffs) for name, diffs in relative_performance_loss_diff.items()}
        observed_max_avg_improvement = max(0, max(mean_loss_diffs.values())) if mean_loss_diffs else 0
        
        # Bootstrap
        bootstrap_max_avg_improvements = []
        d_series_matrix = np.array([diffs for diffs in relative_performance_loss_diff.values()]).T
        
        for _ in range(n_bootstrap):
            bootstrap_indices = np.random.choice(n_obs, n_obs, replace=True)
            d_bootstrap_sample = d_series_matrix[bootstrap_indices, :]
            
            if d_bootstrap_sample.size > 0:
                mean_d_bootstrap = np.mean(d_bootstrap_sample, axis=0)
                current_bootstrap_max = max(0, max(mean_d_bootstrap)) if len(mean_d_bootstrap) > 0 else 0
            else:
                current_bootstrap_max = 0
                
            bootstrap_max_avg_improvements.append(current_bootstrap_max)
            
        p_value = np.mean(np.array(bootstrap_max_avg_improvements) >= observed_max_avg_improvement)
        
        return observed_max_avg_improvement, p_value, mean_loss_diffs
        
    except Exception:
        return np.nan, np.nan, {}


# ==================== CLASSE PRINCIPALE ====================
class ForecastingFramework:
    """
    Framework per analisi out-of-sample di modelli di forecasting.
    Come un sistema di backtesting per strategie finanziarie in condizioni reali.
    """
    
    def __init__(self, config: Config):
        self.config = config
        self.results_metrics = []
        self.all_series_data = None
        self.target_series_clean = None
        self.exog_data_prepared = {}
        self.all_forecast_results = {}
        
        Path(self.config.PATH_OUTPUT_OOS).mkdir(parents=True, exist_ok=True)
        print(f"\nFramework inizializzato")
        print(f"Output directory: {self.config.PATH_OUTPUT_OOS}")

    def _create_pulse_dummy(self, index: pd.DatetimeIndex, event_date_str: str, 
                           duration_months: int, base_name: str) -> pd.Series:
        """Crea variabili dummy per eventi specifici"""
        event_ts = pd.Timestamp(event_date_str)
        dummy = pd.Series(0, index=index, name=base_name)
        event_mask = (index >= event_ts) & (index < event_ts + pd.DateOffset(months=duration_months))
        dummy[event_mask] = 1
        return dummy

    def load_and_prepare_data(self) -> bool:
        """
        Carica e prepara i dati, gestendo correttamente i duplicati.
        Come preparare dati storici di mercato per backtesting puliti e allineati.
        """
        print("\n" + "="*60)
        print(" FASE 1: CARICAMENTO E PREPARAZIONE DATI")
        print("="*60)
        
        try:
            # Caricamento dati
            print(f"\nCaricamento dati da: {os.path.basename(self.config.FILE_SERIE_STAZIONARIE_IN)}")
            self.all_series_data = pd.read_csv(self.config.FILE_SERIE_STAZIONARIE_IN, index_col=0)
            self.all_series_data.index = pd.to_datetime(self.all_series_data.index)
            
            # CORREZIONE CRITICA: Gestione duplicati nell'indice
            if self.all_series_data.index.duplicated().any():
                n_duplicates = self.all_series_data.index.duplicated().sum()
                print(f"Trovati {n_duplicates} duplicati nell'indice - rimozione in corso...")
                self.all_series_data = self.all_series_data[~self.all_series_data.index.duplicated(keep='last')]
            
            self.all_series_data = self.all_series_data.asfreq('MS')
            print(f"Dati caricati: {self.all_series_data.shape[0]} osservazioni, {self.all_series_data.shape[1]} variabili")
            
            # Preparazione serie target
            if self.config.COL_INFLAZIONE_STAZ not in self.all_series_data.columns:
                print(f"Errore: variabile target '{self.config.COL_INFLAZIONE_STAZ}' non trovata")
                return False
                
            self.target_series_clean = self.all_series_data[self.config.COL_INFLAZIONE_STAZ].dropna()
            print(f"\nSerie target: {self.config.COL_INFLAZIONE_STAZ}")
            print(f"   - Periodo: {self.target_series_clean.index[0].strftime('%Y-%m')} a {self.target_series_clean.index[-1].strftime('%Y-%m')}")
            print(f"   - Osservazioni valide: {len(self.target_series_clean)}")
            
            # Preparazione variabili esogene per ogni modello
            print("\nPreparazione variabili esogene per modello:")
            for model_name, config in self.config.MODELLI_DA_TESTARE.items():
                exog_list = []
                
                # Dummy outliers
                if "exog_vars_base" in config:
                    for dummy_name in config["exog_vars_base"]:
                        if dummy_name == "d_outlier_2022_01_1m":
                            exog_list.append(self._create_pulse_dummy(
                                self.all_series_data.index, '2022-01-01', 1, dummy_name
                            ))
                        elif dummy_name == "d_outlier_2022_10_1m":
                            exog_list.append(self._create_pulse_dummy(
                                self.all_series_data.index, '2022-10-01', 1, dummy_name
                            ))
                
                # Variabili Google Trends con lag
                if "gt_vars_lags" in config:
                    for gt_col, lag in config["gt_vars_lags"].items():
                        if gt_col in self.all_series_data.columns:
                            lagged_series = self.all_series_data[gt_col].shift(lag)
                            lagged_series.name = f"{gt_col}_lag{lag}"
                            exog_list.append(lagged_series)
                
                # Combinazione finale
                if exog_list:
                    exog_df = pd.concat(exog_list, axis=1).astype(float)
                    # Rimozione duplicati se presenti
                    if exog_df.index.duplicated().any():
                        exog_df = exog_df[~exog_df.index.duplicated(keep='last')]
                    self.exog_data_prepared[model_name] = exog_df.reindex(self.all_series_data.index)
                    n_vars = len(exog_list)
                    print(f"   • {model_name}: {n_vars} variabili esogene")
                else:
                    self.exog_data_prepared[model_name] = None
                    print(f"   • {model_name}: nessuna variabile esogene")
                    
            print("\nPreparazione dati completata con successo")
            return True
            
        except Exception as e:
            print(f"\nErrore critico nel caricamento dati: {e}")
            return False

    def _forecast_single_model_oos(self, y_series: pd.Series, exog_series: Optional[pd.DataFrame],
                                  order: Tuple, seasonal_order: Tuple, model_name: str) -> Optional[Dict]:
        """
        Genera previsioni OOS per un singolo modello.
        Come simulare trading in tempo reale usando solo info disponibili al momento.
        """
        forecasts_by_h = {h: [] for h in self.config.ORIZZONTI_PREVISIONE}
        n_obs = len(y_series)
        max_h = max(self.config.ORIZZONTI_PREVISIONE)
        n_iterations = n_obs - self.config.LUNGHEZZA_FINESTRA_INIZIALE - max_h + 1
        
        if n_iterations <= 0:
            return None
            
        print(f"   → Esecuzione {n_iterations} iterazioni di forecasting...")
        successful_iters = 0
        
        # Progress tracking
        start_time = time.time()
        
        for i in range(n_iterations):
            try:
                # Definizione finestra training (rolling vs recursive)
                if self.config.SCHEMA_PREVISIONE == 'rolling':
                    train_start = i
                    train_end = i + self.config.LUNGHEZZA_FINESTRA_INIZIALE - 1
                else:  # recursive
                    train_start = 0
                    train_end = self.config.LUNGHEZZA_FINESTRA_INIZIALE + i - 1
                
                # Estrazione dati training
                y_train = y_series.iloc[train_start:train_end + 1]
                exog_train = exog_series.iloc[train_start:train_end + 1] if exog_series is not None else None
                
                # Stima modello
                model = sm.tsa.SARIMAX(
                    y_train, exog=exog_train, order=order, seasonal_order=seasonal_order,
                    enforce_stationarity=False, enforce_invertibility=False,
                    initialization='approximate_diffuse'
                )
                fitted = model.fit(disp=False, maxiter=200, method='lbfgs')
                
                # Previsioni per ogni orizzonte
                for h in self.config.ORIZZONTI_PREVISIONE:
                    # Preparazione esogene future
                    if exog_series is not None:
                        exog_start = train_end + 1
                        exog_end = train_end + h
                        if exog_end < n_obs:
                            exog_forecast = exog_series.iloc[exog_start:exog_end + 1]
                            if len(exog_forecast) != h:
                                continue
                        else:
                            continue
                    else:
                        exog_forecast = None
                    
                    # Generazione previsione
                    forecast = fitted.get_forecast(steps=h, exog=exog_forecast)
                    forecast_value = forecast.predicted_mean.iloc[-1]
                    
                    # Valore osservato
                    actual_idx = train_end + h
                    if actual_idx < n_obs:
                        forecasts_by_h[h].append({
                            'date': y_series.index[actual_idx],
                            'actual': y_series.iloc[actual_idx],
                            'forecast': forecast_value
                        })
                
                successful_iters += 1
                
                # Progress update
                if (i + 1) % 10 == 0:
                    progress = (i + 1) / n_iterations * 100
                    elapsed = time.time() - start_time
                    eta = elapsed / (i + 1) * (n_iterations - i - 1)
                    print(f"      Progresso: {progress:.1f}% - ETA: {eta:.0f}s", end='\r')
                    
            except Exception:
                continue
        
        print(f"\n      ✓ Completate {successful_iters}/{n_iterations} iterazioni")
        
        # Conversione a DataFrame con gestione duplicati
        forecast_dfs = {}
        for h, preds_list in forecasts_by_h.items():
            if preds_list:
                df = pd.DataFrame(preds_list)
                # CORREZIONE: Rimozione duplicati prima di set_index
                if df['date'].duplicated().any():
                    df = df.drop_duplicates(subset=['date'], keep='last')
                forecast_dfs[h] = df.set_index('date')
                
        return forecast_dfs if forecast_dfs else None

    def generate_all_oos_forecasts(self) -> bool:
        """Genera tutte le previsioni OOS per tutti i modelli"""
        print("\n" + "="*60)
        print("FASE 2: GENERAZIONE PREVISIONI OUT-OF-SAMPLE")
        print(f"   Schema: {self.config.SCHEMA_PREVISIONE} | Finestra: {self.config.LUNGHEZZA_FINESTRA_INIZIALE} mesi")
        print("="*60)
        
        for model_name, model_config in self.config.MODELLI_DA_TESTARE.items():
            print(f"\n📊 Modello: {model_name}")
            print(f"   SARIMA{model_config['order']}{model_config['seasonal_order']}")
            
            # Preparazione dati allineati per il modello
            exog = self.exog_data_prepared.get(model_name)
            if exog is not None:
                # Allineamento e rimozione NA
                combined = pd.concat([self.target_series_clean, exog], axis=1)
                combined_clean = combined.dropna()
                y_model = combined_clean.iloc[:, 0]
                exog_model = combined_clean.iloc[:, 1:]
            else:
                y_model = self.target_series_clean.copy()
                exog_model = None
            
            # Verifica dati sufficienti
            min_obs = self.config.LUNGHEZZA_FINESTRA_INIZIALE + max(self.config.ORIZZONTI_PREVISIONE)
            if len(y_model) < min_obs:
                print(f"     Dati insufficienti: {len(y_model)} obs (min richieste: {min_obs})")
                continue
            
            # Generazione previsioni
            forecasts = self._forecast_single_model_oos(
                y_model, exog_model,
                model_config['order'], model_config['seasonal_order'],
                model_name
            )
            
            if forecasts:
                self.all_forecast_results[model_name] = forecasts
                # Report previsioni generate
                for h, df in forecasts.items():
                    print(f"   • h={h}: {len(df)} previsioni generate")
            else:
                print(f"    Nessuna previsione generata")
        
        success = bool(self.all_forecast_results)
        if success:
            print(f"\nPrevisioni completate per {len(self.all_forecast_results)} modelli")
        else:
            print("\nNessuna previsione generata con successo")
            
        return success

    def calculate_and_store_forecast_metrics(self) -> bool:
        """Calcola metriche di accuratezza delle previsioni"""
        print("\n" + "="*60)
        print("FASE 3: CALCOLO METRICHE DI ACCURATEZZA")
        print("="*60)
        
        self.results_metrics = []
        
        for model_name, forecasts_by_h in self.all_forecast_results.items():
            for h, forecast_df in forecasts_by_h.items():
                if forecast_df is not None and len(forecast_df) >= self.config.MIN_OBS_FORECAST_TESTS // 2:
                    actual = forecast_df['actual'].values
                    forecast = forecast_df['forecast'].values
                    
                    # Calcolo metriche
                    rmse = np.sqrt(mean_squared_error(actual, forecast))
                    mae = mean_absolute_error(actual, forecast)
                    mape = np.mean(np.abs((actual - forecast) / np.where(np.abs(actual) < 1e-9, 1e-9, actual))) * 100
                    
                    # Direzione corretta
                    direction_accuracy = np.mean(np.sign(actual) == np.sign(forecast)) * 100
                    
                    self.results_metrics.append({
                        'Modello': model_name,
                        'Orizzonte': h,
                        'RMSE': rmse,
                        'MAE': mae,
                        'MAPE': mape,
                        'Dir_Accuracy': direction_accuracy,
                        'N_Obs': len(forecast_df)
                    })
        
        if not self.results_metrics:
            print("Nessuna metrica calcolata")
            return False
        
        # Salvataggio metriche
        df_metrics = pd.DataFrame(self.results_metrics)
        metrics_file = os.path.join(self.config.PATH_OUTPUT_OOS, f"metriche_oos_{self.config.SCHEMA_PREVISIONE}.csv")
        df_metrics.to_csv(metrics_file, index=False)
        
        # Visualizzazione tabella formattata
        print("\nTabella Riepilogativa Metriche:")
        pivot_rmse = df_metrics.pivot(index='Modello', columns='Orizzonte', values='RMSE')
        pivot_mae = df_metrics.pivot(index='Modello', columns='Orizzonte', values='MAE')
        
        print("\nRMSE per Orizzonte:")
        print(tabulate(pivot_rmse.round(4), headers='keys', tablefmt='grid'))
        
        print("\nMAE per Orizzonte:")
        print(tabulate(pivot_mae.round(4), headers='keys', tablefmt='grid'))
        
        # Identificazione migliori modelli
        print("\nMigliori Modelli per Orizzonte:")
        for h in self.config.ORIZZONTI_PREVISIONE:
            h_data = df_metrics[df_metrics['Orizzonte'] == h]
            if not h_data.empty:
                best_rmse = h_data.loc[h_data['RMSE'].idxmin()]
                print(f"   h={h}: {best_rmse['Modello']} (RMSE={best_rmse['RMSE']:.4f})")
        
        return True

    def _get_aligned_forecasts(self, model1: str, model2: str, horizon: int) -> Tuple[Optional[np.ndarray], ...]:
        """Allinea previsioni di due modelli per confronto"""
        if not all([
            model1 in self.all_forecast_results,
            model2 in self.all_forecast_results,
            horizon in self.all_forecast_results[model1],
            horizon in self.all_forecast_results[model2]
        ]):
            return None, None, None
            
        df1 = self.all_forecast_results[model1][horizon]
        df2 = self.all_forecast_results[model2][horizon]
        
        # Trova date comuni senza duplicati
        common_dates = df1.index.intersection(df2.index)
        common_dates = common_dates[~common_dates.duplicated()]
        
        if len(common_dates) < self.config.MIN_OBS_FORECAST_TESTS:
            return None, None, None
            
        # Estrai valori allineati
        actual = df1.loc[common_dates, 'actual'].values
        pred1 = df1.loc[common_dates, 'forecast'].values
        pred2 = df2.loc[common_dates, 'forecast'].values
        
        return actual, pred1, pred2

    def perform_advanced_forecast_tests(self) -> bool:
        """Esegue test statistici avanzati"""
        print("\n" + "="*60)
        print("FASE 4: TEST STATISTICI AVANZATI")
        print("="*60)
        
        dm_results = []
        cw_results = []
        enc_results = []
        spa_results = []
        
        model_names = list(self.all_forecast_results.keys())
        benchmark_model = "Base_Outliers"
        
        for h in self.config.ORIZZONTI_PREVISIONE:
            print(f"\nOrizzonte h={h}:")
            
            # Preparazione dati per SPA test
            predictions_dict = {}
            actual_values = None
            
            for model in model_names:
                if h in self.all_forecast_results[model]:
                    df = self.all_forecast_results[model][h]
                    if len(df) >= self.config.MIN_OBS_FORECAST_TESTS:
                        if actual_values is None:
                            actual_values = df['actual']
                            common_idx = df.index
                        else:
                            # Trova indice comune senza duplicati
                            common_idx = common_idx.intersection(df.index)
                            common_idx = common_idx[~common_idx.duplicated()]
                        
                        if len(common_idx) >= self.config.MIN_OBS_FORECAST_TESTS:
                            predictions_dict[model] = df.loc[common_idx, 'forecast']
            
            # SPA Test se abbiamo dati sufficienti
            if len(predictions_dict) > 1 and benchmark_model in predictions_dict:
                actual_spa = actual_values.loc[common_idx].values
                preds_spa = {k: v.values for k, v in predictions_dict.items()}
                
                spa_stat, spa_pval, spa_diffs = spa_test(
                    actual_spa, preds_spa, benchmark_model, 
                    n_bootstrap=self.config.N_BOOTSTRAP_SPA
                )
                
                if not np.isnan(spa_stat):
                    spa_results.append({
                        'Orizzonte': h,
                        'Benchmark': benchmark_model,
                        'SPA_Stat': spa_stat,
                        'P_Value': spa_pval,
                        'Significativo': spa_pval < self.config.ALPHA_LEVEL
                    })
                    sig_text = " Significativo" if spa_pval < self.config.ALPHA_LEVEL else ""
                    print(f"    SPA Test: stat={spa_stat:.4f}, p-value={spa_pval:.4f} {sig_text}")
            
            # Test a coppie
            for model1, model2 in combinations(model_names, 2):
                actual_aligned, pred1, pred2 = self._get_aligned_forecasts(model1, model2, h)
                
                if actual_aligned is None:
                    continue
                
                # Diebold-Mariano Test
                try:
                    dm_stat, dm_pval = dm_test(actual_aligned, pred1, pred2, h=h)
                    dm_results.append({
                        'Orizzonte': h,
                        'Modello_1': model1,
                        'Modello_2': model2,
                        'DM_Stat': dm_stat,
                        'P_Value': dm_pval,
                        'Significativo': dm_pval < self.config.ALPHA_LEVEL,
                        'N_Obs': len(actual_aligned)
                    })
                except Exception:
                    pass
                
                # Clark-West Test (solo se model1 è nested in model2)
                if model1 == benchmark_model:
                    cw_stat, cw_pval = clark_west_test(actual_aligned, pred1, pred2, horizon=h)
                    if not np.isnan(cw_stat):
                        cw_results.append({
                            'Orizzonte': h,
                            'Modello_Base': model1,
                            'Modello_Esteso': model2,
                            'CW_Stat': cw_stat,
                            'P_Value': cw_pval,
                            'Significativo': cw_pval < self.config.ALPHA_LEVEL
                        })
                
                # Encompassing Test
                enc_t, enc_p, enc_beta = encompassing_test(actual_aligned, pred1, pred2)
                if not np.isnan(enc_t):
                    enc_results.append({
                        'Orizzonte': h,
                        'Modello_1': model1,
                        'Modello_2': model2,
                        'Beta': enc_beta,
                        'T_Stat': enc_t,
                        'P_Value': enc_p,
                        'M1_Ingloba_M2': enc_p >= self.config.ALPHA_LEVEL
                    })
        
        # Salvataggio risultati
        base_path = self.config.PATH_OUTPUT_OOS
        
        if dm_results:
            pd.DataFrame(dm_results).to_csv(
                os.path.join(base_path, f"test_diebold_mariano_{self.config.SCHEMA_PREVISIONE}.csv"),
                index=False
            )
            print(f"\nSalvati {len(dm_results)} test Diebold-Mariano")
        
        if cw_results:
            pd.DataFrame(cw_results).to_csv(
                os.path.join(base_path, f"test_clark_west_{self.config.SCHEMA_PREVISIONE}.csv"),
                index=False
            )
            print(f"Salvati {len(cw_results)} test Clark-West")
        
        if enc_results:
            pd.DataFrame(enc_results).to_csv(
                os.path.join(base_path, f"test_encompassing_{self.config.SCHEMA_PREVISIONE}.csv"),
                index=False
            )
            print(f"Salvati {len(enc_results)} test Encompassing")
        
        if spa_results:
            pd.DataFrame(spa_results).to_csv(
                os.path.join(base_path, f"test_spa_{self.config.SCHEMA_PREVISIONE}.csv"),
                index=False
            )
            print(f"Salvati {len(spa_results)} test SPA")
        
        return True

    def generate_final_report_and_plots(self) -> bool:
        """Genera report finale e visualizzazioni"""
        print("\n" + "="*60)
        print("FASE 5: GENERAZIONE REPORT E GRAFICI")
        print("="*60)
        
        if not self.results_metrics:
            print("Nessuna metrica disponibile per il report")
            return False
        
        df_metrics = pd.DataFrame(self.results_metrics)
        
        # Configurazione stile grafici
        sns.set_style("whitegrid")
        colors = sns.color_palette("husl", len(df_metrics['Modello'].unique()))
        
        # 1. Grafico RMSE per orizzonte
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        for i, model in enumerate(df_metrics['Modello'].unique()):
            model_data = df_metrics[df_metrics['Modello'] == model]
            ax1.plot(model_data['Orizzonte'], model_data['RMSE'], 
                    marker='o', linewidth=2.5, markersize=8, 
                    label=model, color=colors[i])
            ax2.plot(model_data['Orizzonte'], model_data['MAE'], 
                    marker='s', linewidth=2.5, markersize=8, 
                    label=model, color=colors[i])
        
        ax1.set_xlabel('Orizzonte di Previsione (mesi)', fontsize=12)
        ax1.set_ylabel('RMSE', fontsize=12)
        ax1.set_title('Root Mean Squared Error per Orizzonte', fontsize=14, fontweight='bold')
        ax1.set_xticks(self.config.ORIZZONTI_PREVISIONE)
        ax1.legend(loc='best', framealpha=0.9)
        ax1.grid(True, alpha=0.3)
        
        ax2.set_xlabel('Orizzonte di Previsione (mesi)', fontsize=12)
        ax2.set_ylabel('MAE', fontsize=12)
        ax2.set_title('Mean Absolute Error per Orizzonte', fontsize=14, fontweight='bold')
        ax2.set_xticks(self.config.ORIZZONTI_PREVISIONE)
        ax2.legend(loc='best', framealpha=0.9)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        metrics_plot = os.path.join(self.config.PATH_OUTPUT_OOS, f"metriche_confronto_{self.config.SCHEMA_PREVISIONE}.png")
        plt.savefig(metrics_plot, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Grafico metriche salvato: {os.path.basename(metrics_plot)}")
        
        # 2. Heatmap delle performance relative
        pivot_rmse = df_metrics.pivot(index='Modello', columns='Orizzonte', values='RMSE')
        
        # Calcolo performance relativa rispetto al benchmark
        if 'Base_Outliers' in pivot_rmse.index:
            relative_perf = (pivot_rmse - pivot_rmse.loc['Base_Outliers']) / pivot_rmse.loc['Base_Outliers'] * 100
            
            plt.figure(figsize=(10, 6))
            sns.heatmap(relative_perf, annot=True, fmt='.1f', cmap='RdYlGn_r', center=0,
                       cbar_kws={'label': 'Performance Relativa (%)'})
            plt.title('Performance Relativa vs Modello Base (RMSE)', fontsize=14, fontweight='bold')
            plt.ylabel('Modello', fontsize=12)
            plt.xlabel('Orizzonte (mesi)', fontsize=12)
            plt.tight_layout()
            
            heatmap_plot = os.path.join(self.config.PATH_OUTPUT_OOS, f"heatmap_performance_{self.config.SCHEMA_PREVISIONE}.png")
            plt.savefig(heatmap_plot, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Heatmap performance salvata: {os.path.basename(heatmap_plot)}")
        
        # 3. Grafico accuratezza direzionale
        if 'Dir_Accuracy' in df_metrics.columns:
            fig, ax = plt.subplots(figsize=(10, 6))
            
            pivot_dir = df_metrics.pivot(index='Modello', columns='Orizzonte', values='Dir_Accuracy')
            x = np.arange(len(self.config.ORIZZONTI_PREVISIONE))
            width = 0.2
            
            for i, model in enumerate(pivot_dir.index):
                offset = (i - len(pivot_dir.index)/2 + 0.5) * width
                ax.bar(x + offset, pivot_dir.loc[model], width, 
                      label=model, alpha=0.8)
            
            ax.set_xlabel('Orizzonte di Previsione (mesi)', fontsize=12)
            ax.set_ylabel('Accuratezza Direzionale (%)', fontsize=12)
            ax.set_title('Accuratezza nella Previsione della Direzione', fontsize=14, fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels(self.config.ORIZZONTI_PREVISIONE)
            ax.legend(loc='best')
            ax.grid(True, alpha=0.3, axis='y')
            ax.set_ylim(0, 100)
            
            plt.tight_layout()
            dir_plot = os.path.join(self.config.PATH_OUTPUT_OOS, f"accuratezza_direzionale_{self.config.SCHEMA_PREVISIONE}.png")
            plt.savefig(dir_plot, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Grafico accuratezza direzionale salvato: {os.path.basename(dir_plot)}")
        
        # 4. Report testuale riassuntivo
        report_lines = []
        report_lines.append("="*60)
        report_lines.append("REPORT FINALE ANALISI OUT-OF-SAMPLE")
        report_lines.append("="*60)
        report_lines.append(f"\nData Analisi: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
        report_lines.append(f"Schema Previsione: {self.config.SCHEMA_PREVISIONE}")
        report_lines.append(f"Finestra Iniziale: {self.config.LUNGHEZZA_FINESTRA_INIZIALE} mesi")
        report_lines.append(f"Orizzonti Testati: {self.config.ORIZZONTI_PREVISIONE}")
        report_lines.append(f"Modelli Analizzati: {len(self.config.MODELLI_DA_TESTARE)}")
        
        report_lines.append("\n" + "-"*40)
        report_lines.append("MIGLIORI MODELLI PER ORIZZONTE (RMSE)")
        report_lines.append("-"*40)
        
        for h in self.config.ORIZZONTI_PREVISIONE:
            h_data = df_metrics[df_metrics['Orizzonte'] == h]
            if not h_data.empty:
                best = h_data.loc[h_data['RMSE'].idxmin()]
                report_lines.append(f"h={h:2d}: {best['Modello']:20s} RMSE={best['RMSE']:.4f} MAE={best['MAE']:.4f}")
        
        # Salva report
        report_file = os.path.join(self.config.PATH_OUTPUT_OOS, f"report_finale_{self.config.SCHEMA_PREVISIONE}.txt")
        with open(report_file, 'w') as f:
            f.write('\n'.join(report_lines))
        
        print(f"\nReport testuale salvato: {os.path.basename(report_file)}")
        
        # 5. Configurazione esperimento in JSON
        config_summary = {
            'timestamp': datetime.now().isoformat(),
            'configurazione': {
                'schema_previsione': self.config.SCHEMA_PREVISIONE,
                'finestra_iniziale': self.config.LUNGHEZZA_FINESTRA_INIZIALE,
                'orizzonti': self.config.ORIZZONTI_PREVISIONE,
                'alpha_level': self.config.ALPHA_LEVEL,
                'n_bootstrap_spa': self.config.N_BOOTSTRAP_SPA
            },
            'modelli': self.config.MODELLI_DA_TESTARE,
            'risultati_summary': {
                'n_modelli_testati': len(self.all_forecast_results),
                'n_metriche_calcolate': len(self.results_metrics)
            }
        }
        
        config_file = os.path.join(self.config.PATH_OUTPUT_OOS, "configurazione_esperimento.json")
        with open(config_file, 'w') as f:
            json.dump(config_summary, f, indent=4)
        
        print(f"Configurazione salvata: {os.path.basename(config_file)}")
        print("\nReport e grafici completati con successo!")
        
        return True

    def run_complete_analysis(self) -> bool:
        """Esegue l'analisi completa"""
        print("\n" + "="*20)
        print("ANALISI OUT-OF-SAMPLE PROFESSIONALE")
        print("Framework per Valutazione Modelli di Forecasting")
        print("="*20)
        
        try:
            # Fase 1: Caricamento dati
            if not self.load_and_prepare_data():
                return False
            
            # Fase 2: Generazione previsioni
            if not self.generate_all_oos_forecasts():
                return False
            
            # Fase 3: Calcolo metriche
            if not self.calculate_and_store_forecast_metrics():
                return False
            
            # Fase 4: Test statistici
            self.perform_advanced_forecast_tests()
            
            # Fase 5: Report finale
            self.generate_final_report_and_plots()
            
            print("\n" + "="*60)
            print("ANALISI COMPLETATA CON SUCCESSO!")
            print(f"Tutti i risultati salvati in: {self.config.PATH_OUTPUT_OOS}")
            print("="*60)
            
            return True
            
        except Exception as e:
            print(f"\nErrore critico: {e}")
            import traceback
            traceback.print_exc()
            return False


# ==================== ESECUZIONE PRINCIPALE ====================
def main():
    """Funzione principale"""
    try:
        # Inizializzazione
        config = Config()
        framework = ForecastingFramework(config)
        
        # Esecuzione analisi
        start_time = time.time()
        success = framework.run_complete_analysis()
        elapsed_time = time.time() - start_time
        
        if success:
            print(f"\nTempo totale di esecuzione: {elapsed_time:.1f} secondi")
        else:
            print("\nAnalisi terminata con errori")
            
    except KeyboardInterrupt:
        print("\n\nEsecuzione interrotta dall'utente")
    except Exception as e:
        print(f"\nErrore fatale: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()



Framework inizializzato
Output directory: /Users/tommaso/Desktop/tesi-inflation-gt/SARIMAX_modelli/previsioni_out_of_sample_v3_avanzata

ANALISI OUT-OF-SAMPLE PROFESSIONALE
Framework per Valutazione Modelli di Forecasting

 FASE 1: CARICAMENTO E PREPARAZIONE DATI

Caricamento dati da: indici_gt_nic_stazionari_fase2.csv
Dati caricati: 252 osservazioni, 3 variabili

Serie target: NIC_destag_ISTAT_diff1
   - Periodo: 2004-02 a 2024-12
   - Osservazioni valide: 251

Preparazione variabili esogene per modello:
   • Base_Outliers: 2 variabili esogene
   • GT_Tematico_L1: 3 variabili esogene
   • GT_Inflazione_L3: 3 variabili esogene
   • GT_Entrambi: 4 variabili esogene

Preparazione dati completata con successo

FASE 2: GENERAZIONE PREVISIONI OUT-OF-SAMPLE
   Schema: rolling | Finestra: 180 mesi

📊 Modello: Base_Outliers
   SARIMA(1, 0, 1)(0, 0, 0, 12)
   → Esecuzione 60 iterazioni di forecasting...
      Progresso: 100.0% - ETA: 0s
      ✓ Completate 60/60 iterazioni
   • h=1: 60 previsio